In [23]:
import pubchempy as pcp

# c_list = pcp.get_compounds("C6H12", "formula", record_type="3d")
search_name = "propane"
c_list = pcp.get_compounds(search_name, "name", record_type="3d")

c_list_clean = []
for c in c_list:
    if c.cid < 100000:
        c_list_clean.append(c)

for c in c_list_clean:
    print(c.cid, c.iupac_name, c.synonyms[0])
# c_list_clean.pop()
# for c in c_list_clean:
#     print(c.cid, c.iupac_name, c.synonyms[0])

6334 None PROPANE


In [24]:
from pyscf.geomopt.geometric_solver import optimize
from pyscf import gto, scf
import json
import copy
import json

with open("../cadft/utils/mol.json") as f:
    data = json.load(f)

dict_ = {}

for c in c_list_clean:
    name = search_name
    # if name is None:
    #     name = c.synonyms[0]
    # if name is None:
    #     print("No name for", c.cid)
    #     continue
    name = name.lower().replace(" ", "_")

    molecular = []
    for c_atom in c.atoms:
        atom = []
        atom.append(c_atom.element)
        atom.append(round(c_atom.x, 4))
        atom.append(round(c_atom.y, 4))
        atom.append(round(c_atom.z, 4))
        molecular.append(atom)

    mol = gto.M(atom=molecular, basis="ccpvdz")
    mdft = mol.KS()
    mdft.xc = "b3lyp"
    mdft.kernel()
    mol_eq = optimize(mdft, maxsteps=100, verbose=0)

    molecular_new = copy.deepcopy(molecular)
    for i, atom in enumerate(molecular_new):
        atom[1] = mol_eq.atom_coords(unit="ANG")[i][0]
        atom[2] = mol_eq.atom_coords(unit="ANG")[i][1]
        atom[3] = mol_eq.atom_coords(unit="ANG")[i][2]

    dict_[name] = molecular

data.update(dict_)

with open("../cadft/utils/mol.json", "w") as f:
    json.dump(data, f)

converged SCF energy = -119.143202869191


geometric-optimize called with the following command line:
/home/dhem/anaconda3/envs/pyscf/lib/python3.11/site-packages/ipykernel_launcher.py --f=/home/dhem/.local/share/jupyter/runtime/kernel-v2-29882lbbzwlRwJVY.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              *****


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000  -0.568900   0.000000    0.000000  0.000000  0.000000
   C  -1.257100   0.284400   0.000000    0.000000  0.000000  0.000000
   C   1.257100   0.284500   0.000000    0.000000  0.000000  0.000000
   H   0.000000  -1.218300   0.882400    0.000000  0.000000  0.000000
   H   0.000000  -1.218300  -0.882400    0.000000  0.000000  0.000000
   H  -1.296900   0.924400   0.887300    0.000000  0.000000  0.000000
   H  -1.296700   0.924500  -0.887200    0.000000 -0.000000  0.000000
   H  -2.147500  -0.352000  -0.000100    0.000000  0.000000  0.000000
   H   2.147500  -0.352000   0.000000    0.000000  0.000000  0.000000
   H   1.296800   0.924500   0.887200    0.000000 -0.000000  0.000000
   H   1.296800   0.924500  -0.887200    0.000000 -0.000000  0.000000
converged SCF energy = -119.143202869175
--------------- RKS_Scanner gradients ---------------
       

Step    0 : Gradient = 5.994e-03/6.479e-03 (rms/max) Energy = -119.1432028692
Hessian Eigenvalues: 2.30000e-02 2.30000e-02 5.00000e-02 ... 3.42786e-01 3.42970e-01 3.43036e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000007  -0.566211   0.000003    0.000007  0.002689  0.000003
   C  -1.281137   0.279418  -0.000009   -0.024037 -0.004982 -0.000009
   C   1.281133   0.279453   0.000006    0.024033 -0.005047  0.000006
   H   0.000029  -1.228158   0.884568    0.000029 -0.009858  0.002168
   H   0.000026  -1.228157  -0.884562    0.000026 -0.009857 -0.002162
   H  -1.328107   0.929708   0.890177   -0.031207  0.005308  0.002877
   H  -1.328000   0.929842  -0.890103   -0.031300  0.005342 -0.002903
   H  -2.187469  -0.349336  -0.000099   -0.039969  0.002664  0.000001
   H   2.187461  -0.349308   0.000009    0.039961  0.002692  0.000009
   H   1.328026   0.929793   0.890156    0.031226  0.005293  0.002956
   H   1.328030   0.929791  -0.890146    0.031230  0.005291 -0.002946

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess d

...

Step    2 : Displace = 4.287e-03/6.176e-03 (rms/max) Trust = 1.414e-01 (+) Grad = 4.402e-04/1.109e-03 (rms/max) E (change) = -119.1440766503 (-3.013e-05) Quality = 1.144
Hessian Eigenvalues: 2.30000e-02 2.30000e-02 4.99995e-02 ... 3.42786e-01 3.43003e-01 3.46177e-01



Geometry optimization cycle 4
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000005  -0.565362  -0.000003    0.000000  0.000873 -0.000003
   C  -1.276479   0.279390  -0.000007    0.000383 -0.000363  0.000000
   C   1.276474   0.279415   0.000008   -0.000384 -0.000367  0.000002
   H   0.000007  -1.230811   0.882322   -0.000005 -0.000444 -0.000950
   H   0.000018  -1.230806  -0.882331   -0.000002 -0.000441  0.000942
   H  -1.323861   0.931435   0.889368    0.000162  0.000197 -0.000494
   H  -1.323782   0.931545  -0.889306    0.000149  0.000184  0.000486
   H  -2.180954  -0.350797  -0.000086    0.000471 -0.000166  0.000008
   H   2.180961  -0.350754   0.000014   -0.000466 -0.000163  0.000002
   H   1.323798   0.931515   0.889347   -0.000156  0.000191 -0.000491
   H   1.323810   0.931517  -0.889329   -0.000153  0.000194  0.000497
converged SCF energy = -119.144079192762
--------------- RKS_Scanner gradients ---------------
       

Step    3 : Displace = 6.877e-04/1.037e-03 (rms/max) Trust = 2.000e-01 (+) Grad = 9.408e-05/2.378e-04 (rms/max) E (change) = -119.1440791928 (-2.542e-06) Quality = 1.160
Hessian Eigenvalues: 2.29996e-02 2.30000e-02 4.28841e-02 ... 3.42786e-01 3.43004e-01 3.55498e-01



Geometry optimization cycle 5
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000006  -0.565094  -0.000004    0.000000  0.000268 -0.000002
   C  -1.276645   0.279280  -0.000007   -0.000166 -0.000110 -0.000000
   C   1.276640   0.279305   0.000009    0.000166 -0.000111  0.000001
   H   0.000006  -1.231047   0.882027   -0.000001 -0.000236 -0.000295
   H   0.000018  -1.231039  -0.882042    0.000000 -0.000232  0.000289
   H  -1.324215   0.931517   0.889243   -0.000354  0.000082 -0.000126
   H  -1.324145   0.931616  -0.889189   -0.000363  0.000071  0.000117
   H  -2.181108  -0.350887  -0.000078   -0.000153 -0.000090  0.000008
   H   2.181115  -0.350844   0.000015    0.000154 -0.000090  0.000001
   H   1.324155   0.931590   0.889227    0.000357  0.000075 -0.000119
   H   1.324169   0.931596  -0.889204    0.000359  0.000079  0.000125
converged SCF energy = -119.144079345126
--------------- RKS_Scanner gradients ---------------
       

Step    4 : Displace = 3.138e-04/3.946e-04 (rms/max) Trust = 2.828e-01 (+) Grad = 9.932e-06/1.741e-05 (rms/max) E (change) = -119.1440793451 (-1.524e-07) Quality = 0.959
Hessian Eigenvalues: 2.29996e-02 2.30000e-02 4.28841e-02 ... 3.42786e-01 3.43004e-01 3.55498e-01
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 27.101 seconds


In [1]:

import importlib.resources
from pathlib import Path
import os
import json

with importlib.resources.path("cadft", "utils") as resource_path:
    with open(
        Path(os.fspath(resource_path)) / "mol.json",
        "r",
        encoding="utf-8",
    ) as f:
        Mol = (json.load(f))
print(Mol.keys())

/tmp/ipykernel_15208/2901700228.py:6: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with importlib.resources.path("cadft", "utils") as resource_path:


MAIN_PATH: /home/dhem/workspace/2024.1
DATA_PATH: /home/dhem/workspace/2024.1/data/grids_mrks
DATA_SAVE_PATH: /home/dhem/workspace/2024.1/data/grids_mrks/saved_data
DATA_CC_PATH: /home/dhem/workspace/2024.1/data/test
dict_keys(['hexane', 'Cyclopentane', '2-methylpentane', '3-methylpentane', '2,2-dimethylbutane', '2,3-dimethylbutane', 'cyclohexane', '1-hexene', 'methylcyclopentane', '3,3-dimethyl-1-butene', '4-methyl-1-pentene', '2-methyl-1-pentene', '2-hexene', '2,3-dimethyl-2-butene', '3-hexene', '2-ethyl-1-butene', 'propylcyclopropane', '2,3-dimethyl-1-butene', '2-methyl-2-pentene', '4-methyl-2-pentene', '3-methyl-1-pentene', '2-pentene, 3-methyl-', '1-ethyl-1-methylcyclopropane', 'methane', 'ethane', 'ethylene', 'acetylene', 'cyclopentane', 'neopentane', 'isopentane', 'pentane', 'benzene', 'spiropentane', 'cyclobutane', 'cyclopropylmethyl', 'butyne', 'butadiene', 'butane', 'isobutane', 'propane'])


2,3-dimethyl-2-butene 3-hexene 2-ethyl-1-butene propylcyclopropane 2,3-dimethyl-1-butene 2-methyl-2-pentene 4-methyl-2-pentene 3-methyl-1-pentene 2-pentene